INSTALAÇÃO DE DEPENDÊNCIAS E BIBLIOTECAS QUE SERÃO NECESSÁRIAS

In [ ]:
!pip install pydantic
!pip install pydantic[email]

IMPORTAÇÕES DAS BIBLIOTECAS QUE SERÃO UTILIZADAS NO CÓDIGO

In [6]:
from enum import auto, IntFlag
from typing import Any
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    ValidationError,
)

CRIAÇÃO DE UMA CLASSE DE CLASSIFICAÇÃO DE PERMISSÕES

In [ ]:
# Essa classe serve para designar uma função para o Usuário
# O ordem de "importância" segue a ordem de declaração
# Ou seja, Author < Editor < Developer < Admin
# O auto serve para dar uma valor associado à uma potência de 2 para cada função
class Role(IntFlag):
    Author = auto()
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer

CRIAÇÃO DA CLASSE USUÁRIO

In [ ]:
class User(BaseModel):
    # O uso do 'str' é para garantir que o campo seja uma string, além disso ele oferce um exemplo de um valor válido para essa variável
    name: str = Field(examples=["Arjan"])
    # O uso do 'EmailStr' é para garantir que o campo seja um email
    # O 'Field' é utilizado para adicionar metadados e resrições ao campo, servindo como uma indicação do formato esperado para esse campo
    email: EmailStr = Field(
        # Ele oferce um exemplo de um valor válido para essa variável
        examples=["example@arjancodes.com"],
        # Aqui é feita uma descrição do que é esperado para esse campo
        description="The email address of the user",
        #O parâmetro Frozen é utilizado para garantir que o objeto não seja alterado após a sua criação
        frozen=True,
    )
    # O uso do 'SecretStr' é para garantir que o campo seja uma senha, ou seja, caso ela seja exibida em algum lugar, ela será exibida como '********'
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    # O atributo role é utilizado para designar a função do usuário
    role: Role = Field(default=None, description="The role of the user")

CRIAÇÃO DE UMA FUNÇÃO DE VALIDAÇÃO

In [9]:
# A criação da função 'model_validate' é criada com um parametro 'data' que é do tipo 'dict' e não retorna nada
def validate(data: dict[str, Any]) -> None:
    try:
        # Aqui é feita a validação do modelo 'User', se os dados estão de acordo com a estrutura do User
        user = User.model_validate(data)
        # Caso os dados estejam corretos, o usuário é exibido
        print(user)
    except ValidationError as e:
        print('User is invalid')
        # Caso os dados estejam incorretos, os erros são exibidos e é mostrado que o usuário é inválido
        for error in e.errors():
            print(error)

CRIAÇÃO DA FUNÇÃO MAIN (PRINCIPAL)

In [ ]:
def main() -> None:
    """
    Essa função é responsável por chamar a função 'validate' com dados corretos e incorretos
    Ela possui dados de acordo com o que foi pedido e
    dados que não estão de acordo com o que foi pedido
    Exemplo:
     good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
     validate(good_data)


    main() -> name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
    
    """
    good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
    bad_data = {"email": "<bad data>", "password": "<bad data>"}

    # Aqui é chamado a função 'validate' com os dados corretos e incorretos
    validate(good_data)
    validate(bad_data)

EXECUÇÃO DA FUNÇÃO MAIN 

In [11]:
if __name__ == "__main__":
    main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.8/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}
